This notebook performs a comprehensive becnhmark suit for the inference procedure for the double pendulum system using the RxInfer framework.

_Author: Dmitry Bagaev_

In [ ]:
using DrWatson

In [ ]:
@quickactivate "RxInferThesisExperiments"

In [ ]:
using RxInferThesisExperiments, ForneyLab, StaticArrays, Plots, PGFPlotsX, LaTeXStrings
using LinearAlgebra, StableRNGs, Random, BenchmarkTools, ColorSchemes, Dates, DataFrames

In [ ]:
const bfolder = datadir("nlds", "forneylab")

In [ ]:
# Pregenerate paths for benchmark data
mkpath(bfolder);

In [ ]:
# Create default environment with default parameters
const environment = DoublePendulum()

In [ ]:
# Define state-transition function, uses RK4 method internally, see the `src/` folder
f(state) = state_transition(environment)(state)

In [ ]:
# Include the model specification
include(srcdir("models", "forneylab", "doublependulum.jl"));

In [ ]:
function run_benchmark(params)
    @unpack T, niterations, seed = params
    
    states, observations = rand(StableRNG(seed), environment, T);
    model    = double_pendulum(T; force = false);
    result   = run_inference(model, observations; iterations = niterations)
    e_states = extract_posteriors(T, result)
    amse     = compute_amse(states, e_states)
    
    benchmark_modelcreation = @benchmark double_pendulum($T; force = true);
    
    benchmark_inference = @benchmark run_inference(model, observations; iterations = $niterations) setup=begin
        model = double_pendulum($T; force = false);
        states, observations = rand(StableRNG($seed), environment, $T);
    end
    
    emse = compute_emse(seed) do _seed
        local states, observations = rand(StableRNG(_seed), environment, T; random_start = true);
        local model    = double_pendulum(T; force = false);
        local result   = run_inference(model, observations; iterations = niterations)
        local e_states = extract_posteriors(T, result)
        return compute_amse(states, e_states)
    end
    
    output = @strdict T niterations seed states e_states observations amse emse benchmark_modelcreation benchmark_inference
    
    return output
end

In [ ]:
# Here we create a list of parameters we want to run our benchmarks with
benchmark_params = dict_list(Dict(
    "T"           => [ 10, 20, 30, 50, 100 ],
    "niterations" => [ 3 ],
    "seed"        => [ 42 ]
));

In [ ]:
# First run maybe slow, you may track the progress in the terminal
# Subsequent runs will not create new benchmarks 
# but will reload it from data folder
benchmarks = map(benchmark_params) do params
    result, _ = produce_or_load(run_benchmark, bfolder, params; tag = false, force = false)
    return result
end;

In [ ]:
sort(prepare_benchmarks_table(bfolder), [ :T ])

# Versions

In [ ]:
versioninfo()

In [ ]:
] status